In [100]:
import pandas as pd

from pathlib import Path
from pycparser import c_parser,c_ast
from gensim.models.word2vec import Word2Vec

sys.path.append('..')
from tree import ASTNode, SingleNode 
from prepare_data import get_blocks

In [101]:
# code_path  = Path('data/c/example.txt')
# code_path.write_text(code)
code_path  = Path('data/c/example.txt')
code = code_path.read_text()
print(code)

int main()
{
    int i,n,sum;
    scanf("%d",n);
    sum = 0;
    for(i=0;i!=n;i++)
    {
        sum = sum + i;
        if(sum % 2 == 0){
            printf("sum is a  even number");
        }
    }
    printf("%d",sum);
    return 0;
}



### 解析代码段为ast 并 visualize(serialize)

In [102]:
parser = c_parser.CParser()

ast = parser.parse(code)
#print(ast)
ast.show()


FileAST: 
  FuncDef: 
    Decl: main, [], [], []
      FuncDecl: 
        TypeDecl: main, []
          IdentifierType: ['int']
    Compound: 
      Decl: i, [], [], []
        TypeDecl: i, []
          IdentifierType: ['int']
      Decl: n, [], [], []
        TypeDecl: n, []
          IdentifierType: ['int']
      Decl: sum, [], [], []
        TypeDecl: sum, []
          IdentifierType: ['int']
      FuncCall: 
        ID: scanf
        ExprList: 
          Constant: string, "%d"
          ID: n
      Assignment: =
        ID: sum
        Constant: int, 0
      For: 
        Assignment: =
          ID: i
          Constant: int, 0
        BinaryOp: !=
          ID: i
          ID: n
        UnaryOp: p++
          ID: i
        Compound: 
          Assignment: =
            ID: sum
            BinaryOp: +
              ID: sum
              ID: i
          If: 
            BinaryOp: ==
              BinaryOp: %
                ID: sum
                Constant: int, 2
              Const

In [29]:
# 打印ast对象的类名
print(ast.__class__.__name__)
print(isinstance(ast, c_ast.FileAST))
# 根据pycparser/pycparser/c_ast.py文件中的FileAST类的定义，其孩子节点是tuple
# 该tuple的第1个元素是'ext[i]', 第2个元素是ext[i]内容
print(len(ast.children()))

FileAST
True
1


In [42]:
node = ast.children()
print(node[0][0])
print(node[0][1])

ext[0]
FuncDef(decl=Decl(name='main',
                  quals=[
                        ],
                  storage=[
                          ],
                  funcspec=[
                           ],
                  type=FuncDecl(args=None,
                                type=TypeDecl(declname='main',
                                              quals=[
                                                    ],
                                              type=IdentifierType(names=['int'
                                                                        ]
                                                                  )
                                              )
                                ),
                  init=None,
                  bitsize=None
                  ),
        param_decls=None,
        body=Compound(block_items=[Decl(name='i',
                                        quals=[
                                              ],
                    

In [123]:
def traverse(node):
    classname = node.__class__.__name__ 
    print(classname)
    if isinstance(node, str) or len(node.children())==0:
        print('leaf node:', node)
    if node.attr_names:
        print('attr:', node.attr_names)
        if 'op' in node.attr_names:
            print(f'op: {node.op}')
    if classname == 'For':
        print(len(node.children()))
        print(node.children()[0][1])
        print(node.children()[1][1])
        print(node.children()[2][1])
        print(node.children()[3][1])
    for child in node.children():
        #print(child[1])
        traverse(child[1])

In [124]:
traverse(ast)

FileAST
FuncDef
Decl
attr: ('name', 'quals', 'storage', 'funcspec')
FuncDecl
TypeDecl
attr: ('declname', 'quals')
IdentifierType
leaf node: IdentifierType(names=['int'
                     ]
               )
attr: ('names',)
Compound
Decl
attr: ('name', 'quals', 'storage', 'funcspec')
TypeDecl
attr: ('declname', 'quals')
IdentifierType
leaf node: IdentifierType(names=['int'
                     ]
               )
attr: ('names',)
Decl
attr: ('name', 'quals', 'storage', 'funcspec')
TypeDecl
attr: ('declname', 'quals')
IdentifierType
leaf node: IdentifierType(names=['int'
                     ]
               )
attr: ('names',)
Decl
attr: ('name', 'quals', 'storage', 'funcspec')
TypeDecl
attr: ('declname', 'quals')
IdentifierType
leaf node: IdentifierType(names=['int'
                     ]
               )
attr: ('names',)
FuncCall
ID
leaf node: ID(name='scanf'
   )
attr: ('name',)
ExprList
Constant
leaf node: Constant(type='string',
         value='"%d"'
         )
attr: ('type', 'valu

In [76]:
node1 = node[0][1]
print(node1.__class__.__name__)
node1.children()[0][1].children()

FuncDef


(('type',
  FuncDecl(args=None,
           type=TypeDecl(declname='main',
                         quals=[
                               ],
                         type=IdentifierType(names=['int'
                                                   ]
                                             )
                         )
           )),)

In [17]:
root= 'data/'
language = 'c'
embedding_size = 128

In [18]:
blocks = []
get_blocks(ast_for_block,blocks)

In [77]:
for b in blocks:
    print(b.token)
    children = b.children
    child_list = [c.token for c in children]
    print(child_list[1])

NameError: name 'blocks' is not defined

In [20]:
'''
from gensim.models.word2vec import Word2Vec
embedding_size = 128
root='data/'
language = 'c'
word2vec = Word2Vec.load(root+language+'/train/embedding_astnode_w2v_' + str(embedding_size)).wv
vocab = word2vec.vocab
max_token = word2vec.syn0.shape[0]
'''

FileNotFoundError: [Errno 2] No such file or directory: 'data/c/train/embedding_astnode_w2v_128'

In [91]:
def tree_to_index(node):
    token = node.token
    result = [vocab[token].index if token in vocab else max_token]
    children = node.children
    for child in children:
        print(f'tree_to_index: a node  child: {child.token}')
        result.append(tree_to_index(child))
    return result

In [13]:
tree = []
for b in blocks:
    print(b.token)
    btree = tree_to_index(b)
    print(btree)
    tree.append(btree)
print(tree)

FuncDef


NameError: name 'tree_to_index' is not defined

In [96]:
# 打印出某个ast有多少个语句树
print(len(tree))
# 找到每个语句树的长度

lens = [len(item) for item in tree]

max_len = max(lens)

print(lens)
print(max_len)

39
[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 1, 2, 1, 3, 4, 1, 2, 3, 3, 1, 2, 1, 4, 3, 3, 4, 3, 2, 1, 1, 1, 3, 2, 1]
4
